In [1]:
import queue
import numpy as np

In [2]:
class Problem(object):
    def __init__(self, start, goal):
        """Initializes the 8 puzzle board with start and goal states."""
        self.start = start
        self.goal = goal
        
    def actions(self, state):
        """
        Returns the list of actions possible in the given state.
        
        We define action in terms of the movement of the blank tile, i.e. the blank tile
        can move either in L, R, U or D direction. So, we return the new location (i, j)
        of the blank tile.
        """
        # find index of blank tile, in this case 0
        i, j = np.where(state == 0)
        i = i[0]
        j = j[0]
        row, col = np.shape(state)
        valid_moves = []  # new location for blank tile
        if i-1 >= 0:
            valid_moves.append((i-1, j))
        if i+1 < row:
            valid_moves.append((i+1, j))
        if j+1 < col:
            valid_moves.append((i, j+1))
        if j-1 >= 0:
            valid_moves.append((i, j-1))
        return valid_moves
    
    def is_goal_state(self, state):
        return np.array_equal(self.goal, state)
    
    def result(self, state, action):
        """Returns the new state when the action is applied to a state."""
        # swap the current location of blank tile with the new location
        if isinstance(state, np.ndarray):
            st = np.copy(state)
        else:
            raise TypeError("Need numpy.ndarray as input format for state.")
        ic, jc = np.where(st == 0)
        ic = ic[0]
        jc = jc[0]
        # tuple unpacking swapping in python, even works for elements in numpy arrays
        st[ic, jc], st[action[0], action[1]] = st[action[0], action[1]], st[ic, jc]
        return st

In [3]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = self.parent.depth + 1
    
    def child_node(self, problem, action):
        """Compute child node given parent node."""
        new_state = problem.result(self.state, action)
        path_cost = self.path_cost + 1
        return Node(new_state, self, action, path_cost)
    
    def __repr__(self):
        return 'NODE[state={}, depth={}]'.format(self.state, self.depth)

    def __hash__(self):
        return hash(''.join(map(str, self.state.flatten())))
    
    def __eq__(self, other):
        return np.array_equal(self.state, other.state)
    
def solution(node):
    """Returns the path, i.e. a sequence of states from start to goal state."""
    res = []
    while node.parent:
        res.append(node.state)
        node = node.parent
    res.append(node.state)
    return res[::-1]

### Uninformed search: BFS

In [4]:
def breadth_first_search(problem):
    """Breadth first search on a graph."""
    node = Node(problem.start)
    if problem.is_goal_state(node.state):
        return solution(node)
    frontier = queue.Queue()
    frontier.put(node)
    explored = set()
    while not frontier.empty():
        node = frontier.get()
        explored.add(node)
        if node.depth <= 10:
            for action in problem.actions(node.state):
                child = node.child_node(problem, action)
                if child not in explored:
                    if problem.is_goal_state(child.state):
                        return solution(child)
                    frontier.put(child)
    return 'Failure'

In [5]:
start = np.array([[0, 1, 3], [4, 2, 5], [7, 8, 6]])
goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
puzzle = Problem(start, goal)

In [6]:
breadth_first_search(puzzle)

[array([[0, 1, 3],
        [4, 2, 5],
        [7, 8, 6]]), array([[1, 0, 3],
        [4, 2, 5],
        [7, 8, 6]]), array([[1, 2, 3],
        [4, 0, 5],
        [7, 8, 6]]), array([[1, 2, 3],
        [4, 5, 0],
        [7, 8, 6]]), array([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 0]])]

### Iterative deepening search

In [7]:
def recursive_dls(node, problem, limit):
    """Recursive helper function for depth limited search."""
    if problem.is_goal_state(node.state):
        return solution(node)
    elif limit == 0:
        return 'Cutoff'
    else:
        cutoff_occured = False
        for action in problem.actions(node.state):
            child = node.child_node(problem, action)
            result = recursive_dls(child, problem, limit-1)
            if result == 'Cutoff':
                cutoff_occured = True
            elif result != 'Failure':
                return result
        if cutoff_occured:
            return 'Cutoff'
        else:
            return 'Failure'
        

def depth_limited_search(problem, limit):
    node = Node(problem.start)
    return recursive_dls(node, problem, limit)


def iterative_deepening_search(problem):
    for depth in range(10):
        result = depth_limited_search(problem, depth)
        if result != 'Cutoff':
            return result

In [8]:
iterative_deepening_search(puzzle)

[array([[0, 1, 3],
        [4, 2, 5],
        [7, 8, 6]]), array([[1, 0, 3],
        [4, 2, 5],
        [7, 8, 6]]), array([[1, 2, 3],
        [4, 0, 5],
        [7, 8, 6]]), array([[1, 2, 3],
        [4, 5, 0],
        [7, 8, 6]]), array([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 0]])]

### Informed Search: A* using heuristics

In [ ]:
def recursive_best_first_search(problem):
    node = Node(problem.start)
    return rbfs(problem, node, infinity)


def rbfs(problem, node, f_limit):
    if problem.is_goal_state(node.state):
        return solution(node)
    successors = []
    for action in problem.actions(node.state):
        successors.add(node.child_node(problem, action))
    if not successors:
        return 'Failure'
    for s in successors:
        s.f = max(s.g + s.h, node.f)
    while True:
        # find lower